<a href="https://colab.research.google.com/github/jisusP/Data_science/blob/main/Proyecto_informes_juan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sacar notas de la similitud del texto.**

In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import os
from natsort import natsorted
import glob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 17.3 MB/s eta 0:00:00


In [48]:
import fitz  # PyMuPDF
from pathlib import Path

# Función para extraer texto de un PDF
def extract_text_from_pdf(pdf_path):
    # Abre el documento PDF
    document = fitz.open(pdf_path)
    text = ""
    # Itera sobre cada página
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# Ruta del directorio
directorio = "/content/"
patron_pdf = "*.pdf"

dicc={}
# Lista de archivos .pdf en el directorio
archivos_pdf = natsorted(glob.glob(directorio + patron_pdf))
for i, j in enumerate(archivos_pdf):
  dicc[i] = extract_text_from_pdf(j)

# Lista de archivos .txt en el directorio de manera individual
archivos_txt = natsorted(Path(directorio).glob(patron_pdf))
# Obtener solo los nombres de los archivos (sin extensión)
nombres_archivos = [archivo.stem for archivo in archivos_txt]

print(dicc)

{0: '                               \n \n \n \n \n \n \n \n \n \nDETERMINACIÓN DE LA ESTEQUIMETRIA DE UNA REACCIÓN \nQUÍMICA   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nAndrea Rojas, Eileen Salas & Michelle Varela  \nMayo 2024. \nUniversidad del Atlántico. \nFacultad de Ingeniería.  \nQuímica General I. \n2 \n \n \n \n \n \n \nResumen \n \nEste informe de laboratorio se basa en un experimento que explora la \nestequiometría de las reacciones químicas. De este modo, determina las cantidades de los \nreactantes y productos presentes en la misma. Durante el experimento, observamos \nmanifestaciones durante la mezcla de volúmenes, como la formación de precipitados. El \nenfoque central de este experimento fue mostrar algunos principios del análisis \ngravimétrico. Además, su utilidad radica en presentar una técnica general para recoger \ncuantitativamente un precipitado y pesarlo. \nDurante la ejecución del experimento, se observó el razonamiento estequiométrico \nen

In [6]:
def preprocess(text):
    #Texto en minúscula.
    text = text.lower()
    #Toma cualquier carácter alfanumérico y lo convierte en un espacio en blanco. Eliminar signos de puntuación.
    text = re.sub(r'\W', ' ', text)
    #Elimina secuencias de espacio en blanco para dejar una sola separación.
    text = re.sub(r'\s+', ' ', text)
    #tokeniza el texto
    tokens = nltk.word_tokenize(text)
    #Se crea el modelo de lematización.
    lemmatizer = WordNetLemmatizer()
    #Teniendo en cuenta los tokens pasados, se lematizan cada token en la lista.
    #El condicional es para no poner la palabra guía de finalización
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    #Devuelve el texto procesado en una cadena de texto unidos por espacios.
    return ' '.join(tokens)

#preprocess("Había un pato est@acionado 123")

In [7]:
# Ejemplo de cómo estructurar 'all_texts'
reference_report = [dicc[0]]
student_reports = [[pdf_est] for i, pdf_est in dicc.items() if pdf_est not in reference_report]

print(reference_report)
print(student_reports)

# Combina todos los informes en una lista
all_texts = [reference_report] + student_reports
print(all_texts)
#print(all_texts[0])
preprocessed_texts = [preprocess(text[0]) for text in all_texts]

vectorizer = TfidfVectorizer() #se llama el método TF-IDF
X_train = vectorizer.fit_transform(preprocessed_texts) #Se aplica el método TF-IDF en la lista que tiene todos los textos procesados.

['                               \n \n \n \n \n \n \n \n \n \nDETERMINACIÓN DE LA ESTEQUIMETRIA DE UNA REACCIÓN \nQUÍMICA   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nAndrea Rojas, Eileen Salas & Michelle Varela  \nMayo 2024. \nUniversidad del Atlántico. \nFacultad de Ingeniería.  \nQuímica General I. \n2 \n \n \n \n \n \n \nResumen \n \nEste informe de laboratorio se basa en un experimento que explora la \nestequiometría de las reacciones químicas. De este modo, determina las cantidades de los \nreactantes y productos presentes en la misma. Durante el experimento, observamos \nmanifestaciones durante la mezcla de volúmenes, como la formación de precipitados. El \nenfoque central de este experimento fue mostrar algunos principios del análisis \ngravimétrico. Además, su utilidad radica en presentar una técnica general para recoger \ncuantitativamente un precipitado y pesarlo. \nDurante la ejecución del experimento, se observó el razonamiento estequiométrico \nen ca

In [ ]:
def calculate_similarity(student_report, reference_report):
    student_vector = vectorizer.transform([preprocess(student_report[0])]) #Se obtiene el vector TF-IDF del informe del estudiante
    reference_vector = vectorizer.transform([preprocess(reference_report[0])]) #Se obtiene el vector TF-IDF del informe del profesor
    similarity = cosine_similarity(student_vector, reference_vector) #Se le aplica la similitud del coseno a los 2 vectores
    return similarity[0][0]

def similarity_to_grade(similarity):
    return 1 + 4 * similarity

def grade_student_report(student_report):
    similarity = calculate_similarity(student_report, reference_report)
    grade = similarity_to_grade(similarity)
    return grade

In [ ]:
# Ejemplo de uso
for i, report in enumerate(student_reports):
    grade = grade_student_report(report)
    print(f"Calificación del informe del estudiante {i+1}: {grade:.2f}, {nombres_archivos[i+1]}")

Calificación del informe del estudiante 1: 4.53, EXPERIENCIA N° 7
Calificación del informe del estudiante 2: 4.69, Tecnica N°7 Lab
Calificación del informe del estudiante 3: 4.68, informe laboratorio-7


# **Definir la estructura del pdf**

In [10]:
!pip install pyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [60]:
from PyPDF2 import PdfReader, PdfWriter
import re

Estructura = {
        "resumen",
        "palabras clave",
        "1. introducción",
        "2. objetivos",
        "3. materiales y reactivos",
        "3. materiales",
        "4. metodología",
        "5. resultados y discusión",
        "6. conclusión",
        "6. conclusiones",
        "7. bibliografía",
        "8. anexos",
        "8. preguntas"
    }
def nota_estructura(listas):
    num_formato = len(listas)
    positivo = num_formato * 0.1
    negativo = (10-num_formato) * 0.1
    accumulado = positivo - negativo
    return accumulado

def num_ref_intro(lineas):
    # Encontrar los índices de las líneas delimitadas por las palabras clave
    inicio = None
    fin = None
    for i, linea in enumerate(lineas):
        if '1. introducción' in linea:
            inicio = i
        if '2. materiales y metodología.' in linea: #2. objetivos
            fin = i
            #break  # Asumiendo que '2. objetivos' viene después de '1. introducción'
                    #Si los títulos están antes como en la tabla de contenido, con el break, tenderá a tomarlos.

    print(inicio, fin)
    # Extraer la sección de interés
    if inicio is not None and fin is not None and inicio < fin:
        seccion_interes = lineas[inicio:fin]
    else:
        seccion_interes = []
    # Concatenar la sección en un solo texto
    texto_seccion = " ".join(seccion_interes)

    # Utilizar expresiones regulares para encontrar y contar las citas del tipo "[X]"
    patron_cita = re.compile(r'\[\d+\]')
    citas = patron_cita.findall(texto_seccion)

    # Mostrar las citas encontradas y su cantidad
    """print(seccion_interes)
    print(texto_seccion)"""
    print("Citas encontradas:", citas)
    print("Cantidad de citas:", len(citas))

for i_pdf in range(len(archivos_pdf)):
    # Leer el archivo PDF
    pdf_reader = PdfReader(archivos_pdf[i_pdf])

    # Crear una lista para almacenar las líneas
    lineas = []

    # Iterar sobre cada página del PDF
    for page in pdf_reader.pages:
        # Extraer el texto de la página
        texto = page.extract_text()
        if texto:
            # Si hay texto, se procede a hacer el código.
            # Dividir el texto en líneas y agregarlas a la lista
            lineas.extend(texto.splitlines())

    # Imprimir las líneas para verificar
    """for linea in lineas:
        print(linea)

    print(lineas)"""
    #Preprocesamiento de las listas y sus datos.
    # Eliminar espacios en blanco
    lista_sin_espacios = list(filter(None, map(str.strip, lineas)))
    lista_con_saltos_lineas = [elemento.strip() for elemento in lineas]
      #'lineas.strip' elimina todos los espacios en blanco.
      # map() lo que hace es aplicar esa función de un string y eliminarle todos los espacios en blanco a cada elemento de la lista 'lineas'
      #filter(None, ...) filtra todo lo que no sea elementos vacíos, por eso el None.
      #list() convierte los resultados de filter() en una lista.

    print(lista_sin_espacios)

    matches = list(map(str.lower, lista_sin_espacios))
    match_estructure = list(map(lambda x: 1 if x in Estructura else 0, matches))
    formato_encontrado = []

    for k, word in enumerate(lista_con_saltos_lineas):
        if lista_sin_espacios[0] == word:
            if lista_con_saltos_lineas[k+1] != lista_sin_espacios[1]:
                formato_encontrado.append(lista_sin_espacios[0])
                formato_encontrado.append(lista_sin_espacios[1])
            else:
                formato_encontrado.append(lista_sin_espacios[0]+ " "+lista_sin_espacios[1])
                formato_encontrado.append(lista_sin_espacios[2])
    for i,j in enumerate(lista_sin_espacios):
        if match_estructure[i] == 1:
            formato_encontrado.append(j)



    #print(lista_con_saltos_lineas)
    #print(matches)
    #print(match_estructure)
    print(sum(match_estructure))
    print(formato_encontrado)

    print(nota_estructura(formato_encontrado))
    print(num_ref_intro(matches))
    break

['DETERMINACIÓN DE LA ESTEQUIMETRI A DE UNA REACCIÓN', 'QUÍMICA', 'Andrea Rojas, Eileen Salas & Michelle Varela', 'Mayo  2024.', 'Universidad del Atlántico.', 'Facultad de Ingeniería.', 'Química General I.', '2', 'Resumen', 'Este informe de laboratorio se basa en un experimento que explora la', 'estequiometría de las reacciones químicas. De este modo, determina las cantidades de los', 'reactantes y productos presentes en la misma. Durante el experimento, observamos', 'manifestaciones durant e la mezcla de volúmenes, como la formación de precipitados. El', 'enfoque central de este experimento fue mostrar algunos principios del análisis', 'gravimétrico. Además, su utilidad radica en presentar una técnica general para recoger', 'cuantitativamente un prec ipitado y pesarlo.', 'Durante la ejecución del experimento, se observó el razonamiento estequiométrico', 'en cada caso, a partir de los volúmenes relativos de las soluciones, su molaridad y el peso', 'de los precipitados formados. De esta

In [50]:
import re

# Supongamos que esta es tu lista de líneas del PDF
lineass = [
    "1. introducción",
    "Este es el inicio del documento.",
    "Aquí hay una cita [1].",
    "Otro texto en la introducción.",
    "1. introducción. Aquí hay otra cita [2].",
    "2. objetivos",
    "Este es el comienzo de los objetivos.",
    "Otra cita [3] en la sección de objetivos.",
]

# Encontrar los índices de las líneas delimitadas por las palabras clave
inicio = None
fin = None

for i, linea in enumerate(lineass):
    if '1. introducción' in linea.lower():
        inicio = i
    if '2. objetivos' in linea.lower():
        fin = i
        break  # Asumiendo que '2. objetivos' viene después de '1. introducción'

# Extraer la sección de interés
if inicio is not None and fin is not None and inicio < fin:
    seccion_interes = lineass[inicio:fin]
else:
    seccion_interes = []

# Concatenar la sección en un solo texto
texto_seccion = " ".join(seccion_interes)

# Utilizar expresiones regulares para encontrar y contar las citas del tipo "[X]"
patron_cita = re.compile(r'\[\d+\]')
citas = patron_cita.findall(texto_seccion)

# Mostrar las citas encontradas y su cantidad
print("Citas encontradas:", citas)
print("Cantidad de citas:", len(citas))


Citas encontradas: ['[2]']
Cantidad de citas: 1


## Sacar las imágenes del PDF.

In [20]:
import fitz  # PyMuPDF

# Abre el documento PDF

documento = fitz.open(archivos_pdf[0])

# Itera sobre cada página
for numero_pagina in range(len(documento)):
    pagina = documento[numero_pagina]

    # Extrae imágenes
    imagenes = pagina.get_images(full=True)
    for imagen_index, img in enumerate(imagenes):
        xref = img[0]
        base_image = documento.extract_image(xref)
        imagen_bytes = base_image["image"]
        imagen_ext = base_image["ext"]

        # Guarda la imagen en un archivo
        with open(f"imagen_{numero_pagina+1}_{imagen_index+1}.{imagen_ext}", "wb") as imagen_file:
            imagen_file.write(imagen_bytes)
        print(f"Imagen guardada: imagen_{numero_pagina+1}_{imagen_index+1}.{imagen_ext}")


Imagen guardada: imagen_1_1.png
Imagen guardada: imagen_4_1.png
Imagen guardada: imagen_6_1.jpeg
Imagen guardada: imagen_7_1.jpeg
Imagen guardada: imagen_7_2.jpeg
Imagen guardada: imagen_13_1.png
Imagen guardada: imagen_13_2.png
